In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots

In [2]:
path = "C:\\Users\\Tiziano Pacifico\\Desktop\\TESI General\\TESI_FINAL_STAGE\\FASE 6\\DFs\\"
filename = "final_df.json"
df_in = pd.read_json(path+filename, lines=True)

In [3]:
#Divido il dataframe per intervalli giornalieri, riduco le colonne a quelle che mi interessano
df_in['time_utc'] = pd.to_datetime(df_in['utc_datetime_str'])
df_small = df_in[['time_utc','GPT_label','polarity','text']].copy()
day_intervals = pd.date_range(start=df_small['time_utc'].min(), end=df_small['time_utc'].max(), freq='1D')

day_df_list = []
for i in range(len(day_intervals) - 1):
    start_date = day_intervals[i]
    end_date = day_intervals[i + 1]
    week_data = df_small[(df_small['time_utc'] >= start_date) & (df_small['time_utc'] < end_date)].copy()
    day_df_list.append(week_data)
day_df_list.append(df_small[(df_small['time_utc'] >= end_date) & (df_small['time_utc'] <= df_small['time_utc'].max())].copy())

In [5]:
topics = list(df_small['GPT_label'].unique())

In [6]:
#ottengo un dizionario in cui le chiavi sono i topic e i valori sono liste di DFs. Ogni lista contiene i DFs per quei topic divisi per giorni
dfs_per_days_and_topics = {}
for topic in topics:
    days_df_list_per_topic = []
    for dayly_df in day_df_list:
        days_df_list_per_topic.append(dayly_df[dayly_df['GPT_label'] == topic])
    dfs_per_days_and_topics[topic] = days_df_list_per_topic

In [7]:
day_polarity_avg_t = {}
day_pos_polarity_avg_t = {}
day_neg_polarity_avg_t = {}

In [8]:
for topic in topics:
    day_polarity_avg_t[topic] = []
    day_pos_polarity_avg_t[topic] = []
    day_neg_polarity_avg_t[topic] = []
    for df in dfs_per_days_and_topics[topic]:
        day_polarity_avg_t[topic].append(df['polarity'].mean())
        day_pos_polarity_avg_t[topic].append(df['polarity'][df['polarity'] > 0].mean())
        day_neg_polarity_avg_t[topic].append(df['polarity'][df['polarity'] < 0].mean())

In [9]:
volume_post_per_day_t = {}
for topic in topics:
    volume_post_per_day_t[topic] = []
    for topic_dayly_df in dfs_per_days_and_topics[topic]:
        volume_post_per_day_t[topic].append(topic_dayly_df.shape[0])

In [30]:
def plot(avg_list, avg_pos_list, avg_neg_list, volume_list, topic, avg_post_per_day):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
                    vertical_spacing=0.02, 
                    row_heights=[0.2, 0.7])
    fig.add_trace(go.Scatter(x=day_intervals, y=avg_list, mode='lines', name='Polarity Total'), row=2, col=1)
    fig.add_trace(go.Scatter(x=day_intervals, y=avg_pos_list, mode='lines', name='Polarity Positive'), row=2, col=1)
    fig.add_trace(go.Scatter(x=day_intervals, y=avg_neg_list, mode='lines', name='Polarity Negative'), row=2, col=1)

    fig.add_trace(go.Scatter(x=day_intervals, y=volume_list, name='Data Volume'), row=1, col=1)
    fig.add_trace(go.Scatter(x=day_intervals, y=[avg_post_per_day] * len(day_intervals), name='Avg post per day', line = dict(color='black',dash='dash')), row = 1, col = 1)

    fig.update_layout(title=f'Polarity Analysis Over Time with Data Volume topic: {topic}. Avg post per day: {avg_post_per_day}',
                  xaxis_title='Date',
                  yaxis_title='Volume',
                  yaxis2_title='Polarity',
                  showlegend=True)

    fig.show()

In [31]:
avg_post_per_days_per_topic_list = []
for topic in topics:
    avg_post_per_day = (df_small.groupby('GPT_label')['GPT_label'].count()[topic])/len(dfs_per_days_and_topics[topic])
    avg_post_per_days_per_topic_list.append(round(avg_post_per_day))
    plot(day_polarity_avg_t[topic], day_pos_polarity_avg_t[topic], day_neg_polarity_avg_t[topic], volume_post_per_day_t[topic], topic,  round(avg_post_per_day))
